# Competition Notebook 
## Attempt in first week with extensive data cleaning activity... 

In [1]:
!python -V


Python 3.10.13


In [2]:
#install catboost
!pip install -q catboost 

In [3]:
# import libraries
import pandas as pd, gc
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef as mcc


In [4]:
#read in the data
dt = pd.read_csv("/kaggle/input/playground-series-s4e8/train.csv")
dt.set_index("id", drop=True, inplace=True)
dt.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,,
0,e,8.80,f,s,u,f,a,c,w,4.51,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,p,4.51,x,h,o,f,a,c,n,4.79,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,e,6.94,f,s,b,f,x,c,w,6.85,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,e,3.88,f,y,g,f,s,NaN,g,4.16,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,e,5.85,x,l,w,f,d,NaN,w,3.37,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [5]:
# what are the contents of the columns?
print(f"Shape of Data: {dt.shape}")
dt.info()

Shape of Data: (3116945, 21)
<class 'pandas.core.frame.DataFrame'>
Index: 3116945 entries, 0 to 3116944
Data columns (total 21 columns):
 #   Column                Dtype  
---  ------                -----  
 0   class                 object 
 1   cap-diameter          float64
 2   cap-shape             object 
 3   cap-surface           object 
 4   cap-color             object 
 5   does-bruise-or-bleed  object 
 6   gill-attachment       object 
 7   gill-spacing          object 
 8   gill-color            object 
 9   stem-height           float64
 10  stem-width            float64
 11  stem-root             object 
 12  stem-surface          object 
 13  stem-color            object 
 14  veil-type             object 
 15  veil-color            object 
 16  has-ring              object 
 17  ring-type             object 
 18  spore-print-color     object 
 19  habitat               object 
 20  season                object 
dtypes: float64(3), object(18)
memory usage: 523.2+ MB


In [6]:
# list of categoricals to inspect
cat_features = dt.columns.tolist()
cat_features.remove("cap-diameter")
cat_features.remove('stem-height')
cat_features.remove('stem-width')
cat_features.remove('class') # we already fixed this one ;)
cat_features

['cap-shape',
 'cap-surface',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-attachment',
 'gill-spacing',
 'gill-color',
 'stem-root',
 'stem-surface',
 'stem-color',
 'veil-type',
 'veil-color',
 'has-ring',
 'ring-type',
 'spore-print-color',
 'habitat',
 'season']

In [7]:
# count NANs
dt.isna().sum()

class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [8]:
# read in test data
dft = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')
dft.set_index('id',inplace=True, drop=True)
print(f"Test Frame Shape{dft.shape}")
dft.head()

Test Frame Shape(2077964, 20)


,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,
3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,17.12,b,NaN,w,u,w,t,g,NaN,d,a
3116946,6.90,o,t,o,f,NaN,c,y,1.27,10.75,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
3116947,2.00,b,g,n,f,NaN,c,n,6.18,3.14,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3116948,3.47,x,t,n,f,s,c,n,4.98,8.51,NaN,NaN,w,NaN,n,t,z,NaN,d,u
3116949,6.17,x,h,y,f,p,NaN,y,6.73,13.70,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u


In [9]:
# trainframe NANs in categorical features into strings
for i in cat_features:
    dt[i] = dt[i].astype(str)

In [10]:
# testframe categorical NANs to strings in dataframe
for i in cat_features:
    dft[i] = dft[i].astype(str)



In [11]:
# Check to see that the number of unique values in each column match train vs test
for i in cat_features:
    tr = len(dt[i].unique())
    te = len(dft[i].unique())
    print(i, " unique values " , tr , ' vs ', te )

cap-shape  unique values  75  vs  63
cap-surface  unique values  84  vs  60
cap-color  unique values  79  vs  58
does-bruise-or-bleed  unique values  27  vs  23
gill-attachment  unique values  79  vs  67
gill-spacing  unique values  49  vs  36
gill-color  unique values  64  vs  57
stem-root  unique values  39  vs  32
stem-surface  unique values  61  vs  55
stem-color  unique values  60  vs  56
veil-type  unique values  23  vs  16
veil-color  unique values  25  vs  24
has-ring  unique values  24  vs  24
ring-type  unique values  41  vs  37
spore-print-color  unique values  33  vs  34
habitat  unique values  53  vs  40
season  unique values  4  vs  4


In [12]:
# they don't match at all ;) further testing required...
for i in range(len(cat_features)):
    m = cat_features[i]
    print(m)
    testset = dt[m].unique().tolist()
    resset, co = [], 0
    
    for j in dft[m].unique().tolist():
        if j in testset:
            co +=1
        else:
            resset.append(j)
    print(f"{co} good. Missing:", len(resset), resset, '\n')

cap-shape
29 good. Missing: 34 ['3.34', '3.32', '19.76', 'is f', '0.93', 'p f', '1.25', '4.55', '11.43', '6', '5.59', '4.41', '3.73', '3.47', '6.75', '51.63', '6.52', '9.49', '2.7', '3.15', '5.05', 'cap-diameter', '4.51', '4.86', '0.97', '3.94', '6.47', '3.23', '2.02', '1.53', '6.15', '17.89', '0.74', '4.54'] 

cap-surface
29 good. Missing: 31 ['6.04', '10.09', '2.97', '7.78', '5.05', '5.92', '5.98', '11.96', '7.96', '9.53', '17.98', 'habitat', '2.47', '20.64', '8.34', '1.95', '16.48', '5.84', '5.93', '12.3', '14.18', '4.87', '15.52', '3.49', '6.58', '12.15', '6.7', '4.91', '0.94', '2.84', '5.1'] 

cap-color
27 good. Missing: 31 ['10.93', '0.73', '19.18', '5.35', '21.38', 'has-ring', '5.38', '7.41', '18.49', 'habitat', '5.16', '4.5', '4.75', '17.01', '3.0', '24.74', '12.91', '4.11', '4.92', '10.83', '1.83', '2.51', '6.74', '4.18', '23.96', 'cap---------------------------------------------------------------------------------root', '4.96', '0.91', '6.18', '4.61', '9.19'] 

does-bruise-or

In [13]:
#total of all unique values in df and dft
dfc , dftc = 0 , 0
for i in cat_features:
    dfc += len(dt[i].unique())
    dftc +=  len(dft[i].unique())
print("train uniques:", dfc , '\ntest uniques:', dftc )

train uniques: 820 
test uniques: 686


In [14]:
# build some logic to clean them...
def oneshot(val):
    if val in ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'] :
        return val
    else:
        return "NaN"
    
    
def capshape(val):
    if val in ['x','f','s','b','o','p','c','d','e','n','t','w','g','y','r'] :
        #['b','c','x','f','k','s','o','p']
        return val
    else: 
        return "NaN"
    
def capsurface(val):
    if val in ['t','s','y','h','g','d','k','e','i','w','l','n','f','p','u']:
        #['t','g','y','s','h','d','k','e','i','w','l']
        return val
    else: 
        return "NaN"
    
def capcolor(val):
    if val in ['n','y','w', 'g','e','o','p','r','u','b','k','l','f','d','s']:
        #['n','y','w', 'g','e','o','p','r','u','b','k','l']
        return val
    else: 
        return "NaN"
    
def bruises(val):
    if val in ['t','f']:
        return val
    else: 
        return "NaN"
    
    
def odor(val):
    if val in ['t','f']:
        return val
    else: 
        return "NaN"    

def gillattachment(val):
    if val in ['a','d','x','e','s','p','f']:
        return val
    else: 
        return "NaN"
        
def gillspacing(val):
    if val in ['c','f','d']:
        return val
    else: 
        return "NaN"
        
def gillsize(val):
    if val in ['b','n']:
        return val
    else: 
        return "NaN"
        
def gillcolor(val):
    if val in ['w','y','n','p','g','o','k','f','r','e','b','u','l','d','t']:
    #['k','n','b','h','g','r','o','p','u','e','w','y']:
        return val
    else: 
        return "NaN"
        
def stalkroot(val):
    if val in ['b','s','r','c','f','d']:
        #['b','s','r','c','f','d']
        return val
    else: 
        return "NaN"

def stalksurface(val):
    if val in ['s','y','i','t','g','k','h','f','w','d','e','n','x','b','c']:
        #['s','y','i','t','g','k','h','f']
        return val
    else: 
        return "NaN"

def stalkcolor(val):
    if val in ['w','n','y','g','o','e','u','p','k','r','l','b','f','s','t']:
        #['w','n','y','g','o','e','u','p','k','r','l','b','f']
        return val
    else: 
        return "NaN"
        
def ssar(val):
    if val in ['f','y','k','s']:
        return val
    else: 
        return "NaN"
        
def ssbr(val):
    if val in ['f','y','k','s']:
        return val
    else: 
        return "NaN"
        
def scar(val):
    if val in ['n','b','c','g','o','p','e','w','y']:
        return val
    else: 
        return "NaN"
        
def scbr(val):
    if val in ['n','b','c','g','o','p','e','w','y']:
        return val
    else: 
        return "NaN"
        
def veiltype(val):
    if val in ['u']:
        return val
    else: 
        return "NaN"
        
def veilcolor(val):
    if val in ['w','y','n','u','k','e']:
        return val
    else: 
        return "NaN"
        
def ringnumber(val):
    if val in ['f','t']:
        return val
    else: 
        return "NaN"
        
def ringtype(val):
    if val in ['f','e','z','l','r','p','g','m','t','d','n']:
        return val
    else: 
        return "NaN"
    
def spc(val):
    if val in ['k','p','w','n','r','u','g','y']:
        return val
    else: 
        return "NaN"
    
def population(val):
    if val in ['a','c','n','s','v','y']:
        return val
    else: 
        return "NaN"
    
def habitat(val):
    if val in ['d','g','l','m','h','w','p','u','e','s','n','t','r']:
        return val
    else: 
        return "NaN"
    
def season(val):
    if val in ['a','u','w','s']:
        return val
    else: 
        return "NaN"
    


In [15]:
# dt['cap-shape'] = dt['cap-shape'].apply(lambda x : capshape(x))
# dt['cap-surface'] = dt['cap-surface'].apply(lambda x : capsurface(x))
# dt['cap-color'] = dt['cap-color'].apply(lambda x : capcolor(x))
# dt['does-bruise-or-bleed'] = dt['does-bruise-or-bleed'].apply(lambda x : bruises(x))
# dt['gill-attachment'] = dt['gill-attachment'].apply(lambda x : gillattachment(x))
# dt['gill-spacing'] = dt['gill-spacing'].apply(lambda x : gillspacing(x))
# dt['gill-color'] = dt['gill-color'].apply(lambda x : gillcolor(x))
# dt['stem-root'] = dt['stem-root'].apply(lambda x : stalkroot(x))
# dt['stem-surface'] = dt['stem-surface'].apply(lambda x : stalksurface(x))
# dt['stem-color'] = dt['stem-color'].apply(lambda x : stalkcolor(x))
# dt['veil-type'] = dt['veil-type'].apply(lambda x : veiltype(x))
# dt['veil-color'] = dt['veil-color'].apply(lambda x : veilcolor(x))
# dt['has-ring'] = dt['has-ring'].apply(lambda x : ringnumber(x))
# dt['ring-type'] = dt['ring-type'].apply(lambda x : ringtype(x))
# dt['spore-print-color'] = dt['spore-print-color'].apply(lambda x : spc(x))
# dt['habitat'] = dt['habitat'].apply(lambda x : habitat(x))
# dt['season'] = dt['season'].apply(lambda x : season(x))

# dft['cap-shape'] = dft['cap-shape'].apply(lambda x : capshape(x))
# dft['cap-surface'] = dft['cap-surface'].apply(lambda x : capsurface(x))
# dft['cap-color'] = dft['cap-color'].apply(lambda x : capcolor(x))
# dft['does-bruise-or-bleed'] = dft['does-bruise-or-bleed'].apply(lambda x : bruises(x))
# dft['gill-attachment'] = dft['gill-attachment'].apply(lambda x : gillattachment(x))
# dft['gill-spacing'] = dft['gill-spacing'].apply(lambda x : gillspacing(x))
# dft['gill-color'] = dft['gill-color'].apply(lambda x : gillcolor(x))
# dft['stem-root'] = dft['stem-root'].apply(lambda x : stalkroot(x))
# dft['stem-surface'] = dft['stem-surface'].apply(lambda x : stalksurface(x))
# dft['stem-color'] = dft['stem-color'].apply(lambda x : stalkcolor(x))
# dft['veil-type'] = dft['veil-type'].apply(lambda x : veiltype(x))
# dft['veil-color'] = dft['veil-color'].apply(lambda x : veilcolor(x))
# dft['has-ring'] = dft['has-ring'].apply(lambda x : ringnumber(x))
# dft['ring-type'] = dft['ring-type'].apply(lambda x : ringtype(x))
# dft['spore-print-color'] = dft['spore-print-color'].apply(lambda x : spc(x))
# dft['habitat'] = dft['habitat'].apply(lambda x : habitat(x))
# dft['season'] = dft['season'].apply(lambda x : season(x))

In [16]:
dt['cap-shape'] = dt['cap-shape'].apply(lambda x : oneshot(x))
dt['cap-surface'] = dt['cap-surface'].apply(lambda x : oneshot(x))
dt['cap-color'] = dt['cap-color'].apply(lambda x : oneshot(x))
dt['does-bruise-or-bleed'] = dt['does-bruise-or-bleed'].apply(lambda x : oneshot(x))
dt['gill-attachment'] = dt['gill-attachment'].apply(lambda x : oneshot(x))
dt['gill-spacing'] = dt['gill-spacing'].apply(lambda x : oneshot(x))
dt['gill-color'] = dt['gill-color'].apply(lambda x : oneshot(x))
dt['stem-root'] = dt['stem-root'].apply(lambda x : oneshot(x))
dt['stem-surface'] = dt['stem-surface'].apply(lambda x : oneshot(x))
dt['stem-color'] = dt['stem-color'].apply(lambda x : oneshot(x))
dt['veil-type'] = dt['veil-type'].apply(lambda x : oneshot(x))
dt['veil-color'] = dt['veil-color'].apply(lambda x : oneshot(x))
dt['has-ring'] = dt['has-ring'].apply(lambda x : oneshot(x))
dt['ring-type'] = dt['ring-type'].apply(lambda x : oneshot(x))
dt['spore-print-color'] = dt['spore-print-color'].apply(lambda x : oneshot(x))
dt['habitat'] = dt['habitat'].apply(lambda x : oneshot(x))
dt['season'] = dt['season'].apply(lambda x : oneshot(x))

dft['cap-shape'] = dft['cap-shape'].apply(lambda x : oneshot(x))
dft['cap-surface'] = dft['cap-surface'].apply(lambda x : oneshot(x))
dft['cap-color'] = dft['cap-color'].apply(lambda x : oneshot(x))
dft['does-bruise-or-bleed'] = dft['does-bruise-or-bleed'].apply(lambda x : oneshot(x))
dft['gill-attachment'] = dft['gill-attachment'].apply(lambda x : oneshot(x))
dft['gill-spacing'] = dft['gill-spacing'].apply(lambda x : oneshot(x))
dft['gill-color'] = dft['gill-color'].apply(lambda x : oneshot(x))
dft['stem-root'] = dft['stem-root'].apply(lambda x : oneshot(x))
dft['stem-surface'] = dft['stem-surface'].apply(lambda x : oneshot(x))
dft['stem-color'] = dft['stem-color'].apply(lambda x : oneshot(x))
dft['veil-type'] = dft['veil-type'].apply(lambda x : oneshot(x))
dft['veil-color'] = dft['veil-color'].apply(lambda x : oneshot(x))
dft['has-ring'] = dft['has-ring'].apply(lambda x : oneshot(x))
dft['ring-type'] = dft['ring-type'].apply(lambda x : oneshot(x))
dft['spore-print-color'] = dft['spore-print-color'].apply(lambda x : oneshot(x))
dft['habitat'] = dft['habitat'].apply(lambda x : oneshot(x))
dft['season'] = dft['season'].apply(lambda x : oneshot(x))

In [17]:
#total of all unique values in df and dft
dfc2 , dftc2 = 0 , 0
for i in cat_features:
    dfc2 += len(dt[i].unique())
    dftc2 +=  len(dft[i].unique())
print("train uniques:",dfc2 , " difference ",dfc2-dfc  )
print("test uniques:", dftc2 , ' difference', dftc2-dftc )

train uniques: 374  difference  -446
test uniques: 366  difference -320


In [18]:
#separate x an y
Xt = dt.iloc[:,1:]
Yt = dt.iloc[:,0]
print(Xt.shape, Yt.shape)

(3116945, 20) (3116945,)


In [19]:
#separate into kfolds
skf = StratifiedKFold(n_splits=3,random_state=12, shuffle=True)
list_train, list_test = [], []
for i, (train_index, test_index) in enumerate(skf.split(Xt,Yt)):
    list_train.append(train_index)
    list_test.append(test_index)


In [20]:
# Define a range of hyperparameter values to search through


iterations_values = [3100, 3300, 3500, 3700, 3900, 4100]
#[ 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900] # first runs 100, 300 , 500, #second runs 700, 900, 1100,1400, 
depth_values = [   10]
lr_values = [  0.05]

 

In [21]:
gc.collect()

0

In [22]:
# CV hyperparameter grid search
prog = []
bestscore, j = 0 , 0
bestparams = {}
for iterations in iterations_values:
    for depth in depth_values:
        for lr in lr_values:
            model = CatBoostClassifier(iterations = iterations,
                                       depth = depth,
                                       learning_rate = lr,
                                       task_type="GPU",
                                       verbose=True)
            mccscores = []
            for i in range(len(list_train)):
                 
                x_train, y_train = Xt.loc[list_train[i]], Yt.loc[list_train[i]], 
#                x_test , y_test  =  Xt.loc[list_test[i]] , Yt.loc[list_test[i]]
          

                #only select one of these model fit sections...
                model.fit(x_train,y_train,
                          cat_features = cat_features,
                          verbose=False,)
        
#                model.fit(Xt.loc[list_train[i]],Yt.loc[list_train[i]],
#                          cat_features = cat_features,
#                          verbose=False,)


                preds = model.predict(Xt.loc[list_test[i]])
                mcs = mcc(Yt.loc[list_test[i]],preds)
                print(f"Fold {i}: MCC: {mcs}")
                mccscores.append(mcs)
            meanmcc = sum(mccscores) / len(mccscores)
            print(f"Mean of Fold Scores: {meanmcc} Iters: {iterations}  Depth: {depth}  LR: {lr}")
            if meanmcc > bestscore:
                bestscore = meanmcc
                bestparams = {
                    'iterations' : iterations,
                    'depth' : depth,
                    'learning_rate':lr
                }
            
            prog.append({
                'Iterations':iterations,
                'Depth': depth,
                'Learning Rate': lr,
                'Mean MCC': meanmcc
            })
            gc.collect()
            j+=1
            print(f"{j}/10")
                

Fold 0: MCC: 0.9842182430424656
Fold 1: MCC: 0.9849324560483292
Fold 2: MCC: 0.9842560212860427
Mean of Fold Scores: 0.9844689067922792 Iters: 3100  Depth: 10  LR: 0.05
1/10
Fold 0: MCC: 0.9842898028196739
Fold 1: MCC: 0.9849014163121685
Fold 2: MCC: 0.9843064809265137
Mean of Fold Scores: 0.9844992333527854 Iters: 3300  Depth: 10  LR: 0.05
2/10
Fold 0: MCC: 0.9842413770594813
Fold 1: MCC: 0.9849558011609976
Fold 2: MCC: 0.984329824955407
Mean of Fold Scores: 0.9845090010586287 Iters: 3500  Depth: 10  LR: 0.05
3/10
Fold 0: MCC: 0.984276318160465
Fold 1: MCC: 0.9849247469654381
Fold 2: MCC: 0.9843455289525364
Mean of Fold Scores: 0.9845155313594799 Iters: 3700  Depth: 10  LR: 0.05
4/10
Fold 0: MCC: 0.9843094010020426
Fold 1: MCC: 0.984914814071055
Fold 2: MCC: 0.9843413920709806
Mean of Fold Scores: 0.9845218690480261 Iters: 3900  Depth: 10  LR: 0.05
5/10
Fold 0: MCC: 0.9843190268506313
Fold 1: MCC: 0.9848915986948094
Fold 2: MCC: 0.9843081390537923
Mean of Fold Scores: 0.98450625486641

In [23]:
# hyperparameter inspection of results...
# dictionaries to lists...
it , de, lrr, mmcc = [],[],[],[]
for i in range(len(prog)):
        it.append(prog[i]["Iterations"])
        de.append(prog[i]['Depth'])
        lrr.append(prog[i]['Learning Rate'])
        mmcc.append(prog[i]['Mean MCC'])
#data to pandas
dout = pd.DataFrame({'Iters':it,
                    'Depth': de,
                    "LR":lrr,
                    "MC":mmcc})
dout.head()

#sort and new df
ddout = dout.sort_values('MC', ascending=False)
#export sorted to csv
ddout.to_csv("/kaggle/working/thirdset.csv" ,index=True) 
#print best params
print(bestparams)
# visually see
ddout.head()

{'iterations': 3900, 'depth': 10, 'learning_rate': 0.05}


,Iters,Depth,LR,MC
4,3900,10,0.05,0.984522
3,3700,10,0.05,0.984516
2,3500,10,0.05,0.984509
5,4100,10,0.05,0.984506
1,3300,10,0.05,0.984499


In [24]:
ddout

,Iters,Depth,LR,MC
4,3900,10,0.05,0.984522
3,3700,10,0.05,0.984516
2,3500,10,0.05,0.984509
5,4100,10,0.05,0.984506
1,3300,10,0.05,0.984499
0,3100,10,0.05,0.984469


In [25]:
# # in lieu of CV grid search above
# bestparams = {'iterations': 500, 'depth': 10, 'learning_rate': 0.1}

In [26]:
#retrain model on best params
bestmodel = CatBoostClassifier(**bestparams , 
                               cat_features = cat_features, 
                               task_type="GPU",
                               verbose=True)
bestmodel.fit(Xt,Yt)

    

0:	learn: 0.6204294	total: 659ms	remaining: 42m 49s
1:	learn: 0.5422307	total: 1.41s	remaining: 45m 41s
2:	learn: 0.4900117	total: 2.03s	remaining: 43m 54s
3:	learn: 0.4327597	total: 2.86s	remaining: 46m 23s
4:	learn: 0.3814961	total: 3.65s	remaining: 47m 20s
5:	learn: 0.3401081	total: 4.43s	remaining: 47m 53s
6:	learn: 0.3034502	total: 5.24s	remaining: 48m 31s
7:	learn: 0.2729481	total: 6.11s	remaining: 49m 30s
8:	learn: 0.2496030	total: 6.88s	remaining: 49m 33s
9:	learn: 0.2280984	total: 7.68s	remaining: 49m 46s
10:	learn: 0.2102662	total: 8.54s	remaining: 50m 18s
11:	learn: 0.1959901	total: 9.28s	remaining: 50m 7s
12:	learn: 0.1841441	total: 10s	remaining: 50m 2s
13:	learn: 0.1719476	total: 10.7s	remaining: 49m 41s
14:	learn: 0.1618884	total: 11.4s	remaining: 49m 24s
15:	learn: 0.1535149	total: 12.2s	remaining: 49m 23s
16:	learn: 0.1441575	total: 13s	remaining: 49m 33s
17:	learn: 0.1375377	total: 13.7s	remaining: 49m 11s
18:	learn: 0.1304499	total: 14.5s	remaining: 49m 18s
19:	learn

In [27]:
#make predictions
predout = bestmodel.predict(dft)

In [28]:
predout[0:20]

array(['e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'p', 'e', 'p', 'p', 'e',
       'e', 'e', 'p', 'p', 'p', 'p', 'e'], dtype=object)

In [29]:
# add predictions to the test df
dft['preds'] = predout


In [30]:
# split out the part of the df we need and rename the col for submission
predouts = pd.DataFrame(dft.preds.copy())
predouts.columns = ['class'] 
predouts.head()


,class
id,
3116945,e
3116946,p
3116947,p
3116948,p
3116949,e


In [31]:
len(predouts['class'])

2077964

In [32]:

predout2 = predouts.reset_index(drop=False)
predout2.head()

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e


In [33]:
#save submission according to instructions
predout2.to_csv("/kaggle/working/submission.csv", columns=['id','class'], index=False)

In [34]:
# inspect the output
!head /kaggle/working/submission.csv

id,class
3116945,e
3116946,p
3116947,p
3116948,p
3116949,e
3116950,e
3116951,e
3116952,p
3116953,p
